In [1]:
import webotsgym as wg

from webotsgym.config import WebotConfig
from webotsgym.environment import WebotsEnv
from webotsgym.evaluate import Evaluate, EvaluateMats, EvaluatePJ0
from webotsgym.action import DiscreteAction, ContinuousAction
from webotsgym.observation import Observation


import numpy as np
import james

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset
from stable_baselines.gail import generate_expert_traj

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class MyObs(Observation):
    def __init__(self, env):
        super(MyObs, self).__init__(env)
        self.env = env

class MyEval(Evaluate):
    def __init__(self, env, config: WebotConfig = WebotConfig()):
        super(MyEval, self).__init__(env, config)
        self.reward_range = (-100, 100)

    def calc_reward(self):
        if self.env.get_target_distance() < 0.1:
            return self.reward_range[1]
        else:
            dist = self.env.get_target_distance()
            denom = self.env.max_distance / 2
            return -1 * np.tanh(dist / denom)

    def check_done(self):
        if self.env.iterations % self.config.reset_env_after == 0:
            return True
        if self.env.get_target_distance() < 0.25:
            return True
        return False
    

config = WebotConfig()
config.fast_simulation = False
config.reset_env_after = 20000
config.num_obstacles = 0
config.world_size = 4
action_class = ContinuousAction(direction_type="steering", relative=False)

env = WebotsEnv(train=True, 
                action_class=action_class, 
                evaluate_class=MyEval,
                observation_class=MyObs,
                config=config)

/home/ruili/anaconda3/envs/spinningup/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Accepting on Port:  10201
sending: env


In [3]:

def cb(action):
    
    s,r,d,i = env.step((action.heading, action.speed))
    if d is True:
        return



def human_expert(_obs):
    k = james.Keyboard(cb)
    k.action()
    return (k.act.heading, k.act.speed)

n_epi = 1  #number of expert trajectories
expert_name = "expert_human_ppo1_webots_continues_{}".format(n_epi)
###    Remember to touch Esc after everytime you touch Up/Down/Left/Right    ###
generate_expert_traj(human_expert, expert_name, env, n_episodes=n_epi)

#traj_limitation is the number of expert trajectories that you want to use for pretraining
dataset = ExpertDataset(expert_path="{}.npz".format(expert_name), traj_limitation=1, batch_size=128)
env.reset()
model = PPO1("MlpPolicy", env, verbose=1)
model.pretrain(dataset, n_epochs=1000)

sending: env
heading:  0
speed:    0.05
heading:  0.05
speed:    0
heading:  0
speed:    0.05
heading:  0
speed:    -0.05
heading:  0.05
speed:    0
heading:  -0.05
speed:    0
heading:  0
speed:    -0.05
heading:  -0.05
speed:    0
heading:  0
speed:    -0.05
heading:  -0.05
speed:    0
heading:  -0.1
speed:    0
heading:  -0.15000000000000002
speed:    0
heading:  -0.2
speed:    0
heading:  -0.25
speed:    0
heading:  -0.3
speed:    0
heading:  -0.35
speed:    0
heading:  -0.39999999999999997
speed:    0
heading:  -0.44999999999999996
speed:    0
heading:  -0.49999999999999994
speed:    0
heading:  -0.5499999999999999
speed:    0
heading:  -0.6
speed:    0
heading:  -0.65
speed:    0
heading:  -0.7000000000000001
speed:    0
heading:  -0.7500000000000001
speed:    0
heading:  -0.8000000000000002
speed:    0
heading:  -0.8500000000000002
speed:    0
heading:  -0.8500000000000002
speed:    -0.05
heading:  0
speed:    -0.05
heading:  0
speed:    -0.05
heading:  0
speed:    0.05
heading:

Pretraining with Behavior Cloning...
==== Training progress 10.00% ====
Epoch 100
Training loss: 0.029672, Validation loss: 0.077840

==== Training progress 20.00% ====
Epoch 200
Training loss: 0.015063, Validation loss: 0.092232

==== Training progress 30.00% ====
Epoch 300
Training loss: 0.007367, Validation loss: 0.104046

==== Training progress 40.00% ====
Epoch 400
Training loss: 0.002237, Validation loss: 0.107796

==== Training progress 50.00% ====
Epoch 500
Training loss: 0.000962, Validation loss: 0.107143

==== Training progress 60.00% ====
Epoch 600
Training loss: 0.000694, Validation loss: 0.105874

==== Training progress 70.00% ====
Epoch 700
Training loss: 0.000573, Validation loss: 0.104620

==== Training progress 80.00% ====
Epoch 800
Training loss: 0.000478, Validation loss: 0.103612

==== Training progress 90.00% ====
Epoch 900
Training loss: 0.000385, Validation loss: 0.102793

==== Training progress 100.00% ====
Epoch 1000
Training loss: 0.000301, Validation loss: 0

Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ruili/anaconda3/envs/spinningup/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ruili/anaconda3/envs/spinningup/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ruili/anaconda3/envs/spinningup/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ruili/anaconda3/envs/spinningup/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ruili/anaconda3/envs/spinningup/lib/python3.6/site-packages/stable_baselines/gail/dataset/dataset.py", line 290, in _run
    self.queue.put((obs, actions))
  File "/home/ruili/anaconda3/envs/spinningup/lib/python3.6/site-packages/stable_baselines/gail/dataset/dataset.py", line 290, in _run
    self.queue.put((obs, acti

In [4]:
import numpy as np
a = np.load("../rl_testing/actions.npy")
b = np.load("../rl_testing/obs.npy")
print(b.shape)
print(b[1])
print(a.shape)
print(a[1][0])

FileNotFoundError: [Errno 2] No such file or directory: '../rl_testing/actions.npy'